In [1]:
import os
import gc
import tensorflow as tf
import transformers
from transformers import TFBertForSequenceClassification, BertTokenizer

from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
import numpy as np
import pandas as pd
import pickle

import mlflow

2024-11-22 16:35:32.059903: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732293332.074200  543575 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732293332.078647  543575 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-22 16:35:32.092105: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
#mlflow.set_tracking_uri("http://localhost:5000")

In [26]:
# Affichage des versions pour vérification
print("TensorFlow version:", tf.__version__)
print("Transformers version:", transformers.__version__)

TensorFlow version: 2.18.0
Transformers version: 4.46.2


In [25]:
# Recharger le DataFrame depuis le fichier pickle
df_sample = pd.read_pickle('download/df_sample_20000.pkl')
df_sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   target  20000 non-null  int64 
 1   ids     20000 non-null  int64 
 2   date    20000 non-null  object
 3   flag    20000 non-null  object
 4   user    20000 non-null  object
 5   text    20000 non-null  object
dtypes: int64(2), object(4)
memory usage: 937.6+ KB


In [ ]:
 #Vérifier si le DataFrame a au moins 20 000 lignes
if len(df_sample) != 20000:
    raise ValueError("Le DataFrame ne contient pas 20 000 lignes.")

display(df_sample['target'].value_counts())
df = df_sample

target
0    10000
1    10000
Name: count, dtype: int64

# Fine tuning 

### Justification du Fine-Tuning

#### Résultats Prometteurs avec un Modèle Préentraîné

Lors des expérimentations initiales, le modèle préentraîné **`finiteautomata/bertweet-base-sentiment-analysis`** a montré des performances prometteuses sur des tâches similaires d'analyse de sentiments. En effet, ce modèle, conçu spécifiquement pour traiter des données provenant de Twitter, est particulièrement bien adapté pour capturer les nuances linguistiques, les abréviations, et le langage informel souvent présents dans les tweets.

Ces performances encourageantes ont motivé l'idée de pousser davantage l'optimisation en effectuant un **fine-tuning** sur notre corpus spécifique de tweets, dans le but de mieux adapter le modèle aux particularités de nos données et de maximiser la performance sur la tâche ciblée.

En somme, cette approche vise à exploiter le potentiel déjà démontré du modèle tout en le personnalisant davantage pour répondre aux besoins spécifiques de notre problématique métier.


In [ ]:
import re
import emoji
from transformers import AutoTokenizer, TFAutoModel, AutoConfig
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Input, Lambda
from tensorflow.keras.models import Model
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, f1_score, precision_score, recall_score
import mlflow
import mlflow.tensorflow
import time

# Prétraitement des tweets
def preprocess_tweet(tweet):
    tweet = re.sub(r'http\S+|www\S+|https\S+', '', tweet)
    tweet = re.sub(r'@\w+', '', tweet)
    tweet = emoji.demojize(tweet)
    tweet = tweet.lower()
    tweet = re.sub(r'[^a-zA-Z0-9\s]', '', tweet)
    tweet = re.sub(r'\s+', ' ', tweet).strip()
    return tweet

# Nettoyage des tweets
df_sample['cleaned_text'] = df_sample['text'].apply(preprocess_tweet)

# Diviser les données en train et test
documents = df_sample['cleaned_text']
labels = df_sample['target']
X_train, X_val, y_train, y_val = train_test_split(documents, labels, test_size=0.2, random_state=42)

# Charger le tokenizer
model_name = 'finiteautomata/bertweet-base-sentiment-analysis'
#model_name = 'cardiffnlp/twitter-roberta-base-sentiment'
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenisation des données
def tokenize_texts(texts, max_length=128):
    return tokenizer(
        texts.tolist(),
        add_special_tokens=True,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_tensors="tf"
    )

train_encodings = tokenize_texts(X_train, max_length=80)
val_encodings = tokenize_texts(X_val, max_length=80)

# Préparer les datasets TensorFlow
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train.values
)).shuffle(len(X_train)).batch(16)

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    y_val.values
)).batch(16)

# Charger la configuration et le modèle pré-entraîné sans la couche de classification
config = AutoConfig.from_pretrained(model_name, output_hidden_states=False)
base_model = TFAutoModel.from_pretrained(model_name, config=config)

# Dégèle de la dernieère couche du modèle pour accélérer l'entraînement
for layer in base_model.layers[:-1]:
    layer.trainable = False


# Créer un nouveau modèle Keras
input_ids = Input(shape=(80,), dtype=tf.int32, name="input_ids")
attention_mask = Input(shape=(80,), dtype=tf.int32, name="attention_mask")

# Utiliser Lambda pour appeler le modèle pré-entraîné avec output_shape spécifié
sequence_output = Lambda(
    lambda inputs: base_model(input_ids=inputs[0], attention_mask=inputs[1])[0],
    output_shape=(80, 768)
)([input_ids, attention_mask])

cls_token = Lambda(lambda x: x[:, 0, :], output_shape=(768,))(sequence_output)  # Extraire le token [CLS]

# Ajouter une couche dense pour la classification binaire
output = Dense(1, activation="sigmoid")(cls_token)

# Construire le modèle final
model = Model(inputs=[input_ids, attention_mask], outputs=output)

# Optimiseur et perte
learning_rate = 1e-5
optimizer = Adam()
loss = tf.keras.losses.BinaryCrossentropy(from_logits=False)
metric = tf.keras.metrics.BinaryAccuracy('accuracy')

# Compiler le modèle
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

# Intégration avec MLflow
#mlflow.tensorflow.autolog()
# Configuration MLflow
experiment = mlflow.set_experiment("BERT")


with mlflow.start_run(run_name=f"Fine-tuning {model_name}",  nested=True) as run:
    mlflow.set_tag("model_name", model_name)
    mlflow.log_param("epochs", 5)
    mlflow.log_param("batch_size", 16)
    mlflow.log_param("learning_rate", learning_rate)

    # Désactiver l'autolog de TensorFlow
    #mlflow.tensorflow.autolog(disable=True)
    all_validation_metrics = []
    # Entraîner le modèle
    history = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=5
    )

    # Prédictions sur le jeu de validation
    start_time = time.time()
    val_logits = model.predict(val_dataset)
    predict_time = time.time() - start_time
    
    # Convertir les probabilités en classes binaires
    y_val_prob = val_logits.flatten()  # Pour roc_auc_score, garder les probabilités
    y_val_pred = (y_val_prob > 0.5).astype(int) 

    # Calcul des métriques
    validation_metrics = {
        "Accuracy": round(accuracy_score(y_val, y_val_pred), 3),
        "ROC_AUC": round(roc_auc_score(y_val, y_val_prob), 3),
        "F1": round(f1_score(y_val, y_val_pred), 3),
        "Precision": round(precision_score(y_val, y_val_pred), 3),
        "Recall": round(recall_score(y_val, y_val_pred), 3),
        "Predict Time": round(predict_time, 3)
    }

    # Affichage des métriques
    print("Validation Metrics:")
    for metric_name, metric_value in validation_metrics.items():
        print(f"{metric_name}: {metric_value}")

    # Log des métriques dans MLflow
    for metric_name, metric_value in validation_metrics.items():
        mlflow.log_metric(metric_name, metric_value)

    mlflow.log_table(validation_metrics, "3-BERT-ft.json")
    # # Log des métriques finales dans MLflow
    # mlflow.log_metric("Validation Accuracy", accuracy)
    # mlflow.log_dict(
    #     classification_report(y_val, y_val_pred, output_dict=True),
    #     "classification_report.json"
    # )
    run_id = mlflow.active_run().info.run_id
    result = mlflow.register_model(
        model_uri=f"runs:/{run_id}/model",
        name=f"Fine-tuned {model_name}"
    )
    
    # Construire le lien MLflow correspondant (en supposant que vous avez l'URL de votre serveur MLflow)
    active_run = mlflow.active_run()
    run_id = active_run.info.run_id
    #run_name = active_run.data.tags.get("mlflow.runName")  # Obtenir le nom du run depuis les tags

    # Récupérer l'ID de l'expérience active
    experiment_id = active_run.info.experiment_id

    # Définir l'URL de votre serveur MLflow
    mlflow_server_url = "http://localhost:5000"  # Remplacez par l'URL réel de votre serveur MLflow

    # Construire le lien complet vers le run dans l'interface MLflow
    run_link = f"{mlflow_server_url}/#/experiments/{experiment_id}/runs/{run_id}"
    
        # Exemple d'ajout d'un dictionnaire pour chaque configuration
    # Ajouter les métriques de validation à la liste
    all_validation_metrics.append({
        "run_name": f"{model_name} - Fine-tuned",
        **validation_metrics,
        "run_id": run_link
    })
    mlflow.log_table(pd.DataFrame(all_validation_metrics), "3-ft-finiteautomata.json")
    
    mlflow.end_run()


2024-11-22 16:35:44.214349: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
Some layers from the model checkpoint at finiteautomata/bertweet-base-sentiment-analysis were not used when initializing TFRobertaModel: ['classifier']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFRobertaModel were not initialized from the model checkpoint at finiteautomata/bertweet-base-sentiment-analysis and are newly initialized: ['roberta/pooler/dense/bias:0', 'roberta/pooler/

Epoch 1/5


/opt/venv/lib/python3.11/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['input_ids', 'attention_mask']. Received: the structure of inputs={'input_ids': '*', 'token_type_ids': '*', 'attention_mask': '*'}
  warnings.warn(


1000/1000 ━━━━━━━━━━━━━━━━━━━━ 841s 833ms/step - accuracy: 0.4907 - loss: 0.7013 - val_accuracy: 0.5048 - val_loss: 0.6975
Epoch 2/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 783s 783ms/step - accuracy: 0.5028 - loss: 0.6982 - val_accuracy: 0.5048 - val_loss: 0.6932
Epoch 3/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 778s 778ms/step - accuracy: 0.5068 - loss: 0.6993 - val_accuracy: 0.5048 - val_loss: 0.7102
Epoch 4/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 894s 870ms/step - accuracy: 0.4992 - loss: 0.6972 - val_accuracy: 0.5048 - val_loss: 0.7068
Epoch 5/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 833s 833ms/step - accuracy: 0.4999 - loss: 0.6979 - val_accuracy: 0.5048 - val_loss: 0.6950
250/250 ━━━━━━━━━━━━━━━━━━━━ 161s 639ms/step
Validation Metrics:
Accuracy: 0.505
ROC_AUC: 0.501
F1: 0.0
Precision: 0.0
Recall: 0.0
Predict Time: 160.789


/opt/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
Registered model 'Fine-tuned finiteautomata/bertweet-base-sentiment-analysis' already exists. Creating a new version of this model...
2024/11/22 17:47:15 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Fine-tuned finiteautomata/bertweet-base-sentiment-analysis, version 6
Created version '6' of model 'Fine-tuned finiteautomata/bertweet-base-sentiment-analysis'.
2024/11/22 17:47:15 INFO mlflow.tracking._tracking_service.client: 🏃 View run Fine-tuning finiteautomata/bertweet-base-sentiment-analysis at: http://mlflow-server:5000/#/experiments/6/runs/704f7d5b38a14981a3441ce7fbe30595.
2024/11/22 17:47:15 INFO mlfl

# Conclusion sur le Fine-Tuning

Après avoir tenté de fine-tuner le modèle **`finiteautomata/bertweet-base-sentiment-analysis`**, les résultats obtenus n'ont pas permis d'améliorer les performances initiales du modèle pré-entraîné. En effet, les métriques telles que l'**accuracy**, le **F1-score**, la **précision**, et le **rappel** se sont significativement dégradées après le fine-tuning.

Ces résultats suggèrent que le modèle pré-entraîné est déjà très bien ajusté pour la tâche générale d'analyse de sentiments sur des tweets, et qu'il capture efficacement les caractéristiques linguistiques propres à ce type de données. Toutefois, le fine-tuning n'a pas permis d'exploiter pleinement les spécificités de notre corpus.

#### Perspectives d'Amélioration

Pour aller au-delà des performances actuelles, une étude plus approfondie serait nécessaire, incluant :
- Une analyse détaillée des **données d'entraînement**, notamment leur qualité et leur distribution.
- Une exploration systématique des **hyperparamètres** (learning rate, batch size, nombre de couches dégélées, etc.).
- L'intégration de techniques avancées comme l'**augmentation de données** pour enrichir le corpus.
- L'utilisation d'approches comme l'**apprentissage multitâche** ou le **transfer learning sur des tâches adjacentes** pour tirer parti des données disponibles.

En résumé, bien que le modèle pré-entraîné reste performant et adapté à notre cas d'usage, il serait pertinent d'approfondir les analyses et les ajustements pour maximiser son potentiel dans un contexte spécifique.


In [7]:
display(df_sample['target'].value_counts())


target
0    10000
1    10000
Name: count, dtype: int64

# Entrainement modèle BERT
### Expérimentation avec `bert-base-uncased`

Dans le cadre de ce projet, j'ai choisi d'entraîner le modèle **`bert-base-uncased`** sur mon propre jeu de données. Cette expérimentation visait à évaluer si un modèle généraliste pré-entraîné sur un large corpus de textes en anglais pouvait mieux performer que des modèles spécialisés comme **`finiteautomata/bertweet-base-sentiment-analysis`**, qui est spécifiquement conçu pour des données issues de Twitter.

L'objectif était de partir d'un modèle robuste et polyvalent, mais sans préjugés spécifiques liés au domaine ou au type de données (comme les tweets), pour voir s'il pourrait mieux s'adapter à mon corpus grâce à un entraînement complet. Cette démarche a permis de comparer les résultats entre un fine-tuning sur un modèle spécialisé et un entraînement complet sur un modèle généraliste, tout en testant leur capacité respective à capturer les nuances propres à mon jeu de données.

Les résultats obtenus avec **`bert-base-uncased`** serviront également de point de référence pour juger de l'impact du choix du modèle initial et de la qualité de l'entraînement réalisé.


In [8]:

# Charger le tokenizer et le modèle
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
# Exemple de données pour 'documents' et 'labels'
documents = df_sample['text']  # Liste de textes à analyser
labels = df_sample['target']   # Liste de labels (0 ou 1 pour la classification binaire)


In [10]:
# Tokenization des données
def tokenize_data(documents):
    return tokenizer(
        documents.tolist(),
        max_length=128, padding=True, truncation=True, return_tensors='tf'
    )
tokens = tokenize_data(documents)

In [11]:
# Convertir en NumPy pour train_test_split
input_ids_np = tokens['input_ids'].numpy()
attention_masks_np = tokens['attention_mask'].numpy()
labels_np = np.array(labels)

In [12]:
# Séparer les ensembles de données
train_input_ids, val_input_ids, train_labels, val_labels = train_test_split(input_ids_np, labels_np, test_size=0.2, random_state=42)
train_attention_masks, val_attention_masks = train_test_split(attention_masks_np, test_size=0.2, random_state=42)


In [13]:
# Configurer la fonction de perte et l'optimiseur
loss_fn = SparseCategoricalCrossentropy(from_logits=True)
optimizer = Adam(learning_rate=1e-5)

In [14]:
# Définir le nombre d'époques et la taille du batch
epochs = 5
batch_size = 16

# Définir l'accuracy
train_accuracy_metric = tf.keras.metrics.SparseCategoricalAccuracy()

In [15]:
# Configuration MLflow
experiment = mlflow.set_experiment("BERT")

with mlflow.start_run(run_name="BERT", nested=True):
    # Enregistrement des hyperparamètres
    mlflow.log_param("model_name", "bert-base-uncased")
    mlflow.log_param("num_epochs", epochs)
    mlflow.log_param("batch_size", batch_size)
    mlflow.log_param("learning_rate", 1e-5)
    
    # Boucle d'entraînement
    for epoch in range(epochs):
        print(f"\nEpoch {epoch + 1}/{epochs}")
        train_accuracy_metric.reset_state()  # Réinitialiser l'accuracy pour chaque epoch
        epoch_loss = []  # Réinitialiser la liste des pertes pour chaque epoch

        for i in range(0, len(train_input_ids), batch_size):
            # Obtenir un batch de données
            batch_input_ids = train_input_ids[i:i + batch_size]
            batch_attention_masks = train_attention_masks[i:i + batch_size]
            batch_labels = train_labels[i:i + batch_size]
            
            with tf.GradientTape() as tape:
                # Faire des prédictions
                outputs = model(
                    input_ids=batch_input_ids,
                    attention_mask=batch_attention_masks,
                    training=True
                )
                logits = outputs.logits
                
                # Calculer la perte
                loss = loss_fn(batch_labels, logits)
            
            # Calculer et appliquer les gradients
            grads = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))
            
            # Mettre à jour l'accuracy et accumuler la perte
            train_accuracy_metric.update_state(batch_labels, logits)
            epoch_loss.append(loss.numpy())
            
            # Afficher la perte et l'accuracy toutes les 10 batches
            if i % (batch_size * 10) == 0:
                train_accuracy = train_accuracy_metric.result().numpy()
                print(f"Batch {i//batch_size} - Loss: {loss.numpy():.4f} - Accuracy: {train_accuracy:.4f}")
                mlflow.log_metric("batch_train_loss", loss.numpy(), step=i//batch_size)
                mlflow.log_metric("batch_train_accuracy", train_accuracy, step=i//batch_size)
        
        # Enregistrement des métriques de l'époque
        epoch_accuracy = train_accuracy_metric.result().numpy()
        epoch_loss_avg = np.mean(epoch_loss)
        print(f"Epoch {epoch + 1} - Loss: {epoch_loss_avg:.4f} - Accuracy: {epoch_accuracy:.4f}")
        mlflow.log_metric("epoch_train_loss", epoch_loss_avg, step=epoch)
        mlflow.log_metric("epoch_train_accuracy", epoch_accuracy, step=epoch)

    # Libérer la mémoire GPU/CPU non utilisée avant l'évaluation
    gc.collect()
    tf.keras.backend.clear_session()



Epoch 1/5
Batch 0 - Loss: 0.6211 - Accuracy: 0.7500
Batch 10 - Loss: 0.6312 - Accuracy: 0.5682
Batch 20 - Loss: 0.6607 - Accuracy: 0.5952
Batch 30 - Loss: 0.6653 - Accuracy: 0.6008
Batch 40 - Loss: 0.5985 - Accuracy: 0.6113
Batch 50 - Loss: 0.7243 - Accuracy: 0.6140
Batch 60 - Loss: 0.5404 - Accuracy: 0.6168
Batch 70 - Loss: 0.5786 - Accuracy: 0.6276
Batch 80 - Loss: 0.5692 - Accuracy: 0.6427
Batch 90 - Loss: 0.5841 - Accuracy: 0.6449
Batch 100 - Loss: 0.5579 - Accuracy: 0.6498
Batch 110 - Loss: 0.6012 - Accuracy: 0.6548
Batch 120 - Loss: 0.6841 - Accuracy: 0.6643
Batch 130 - Loss: 0.5828 - Accuracy: 0.6751
Batch 140 - Loss: 0.3950 - Accuracy: 0.6826
Batch 150 - Loss: 0.6086 - Accuracy: 0.6858
Batch 160 - Loss: 0.3317 - Accuracy: 0.6949
Batch 170 - Loss: 0.4097 - Accuracy: 0.7036
Batch 180 - Loss: 0.4473 - Accuracy: 0.7082
Batch 190 - Loss: 0.6451 - Accuracy: 0.7114
Batch 200 - Loss: 0.5220 - Accuracy: 0.7170
Batch 210 - Loss: 0.4565 - Accuracy: 0.7207
Batch 220 - Loss: 0.2578 - Accur

2024/11/23 01:40:39 INFO mlflow.tracking._tracking_service.client: 🏃 View run BERT at: http://mlflow-server:5000/#/experiments/6/runs/4c4a6a811ebc4328903983e4cb188778.
2024/11/23 01:40:39 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow-server:5000/#/experiments/6.


In [16]:
from io import BytesIO

def log_plot_to_mlflow(figure, artifact_name):
    """
    Enregistre une figure matplotlib dans MLflow en utilisant un buffer en mémoire.

    :param figure: la figure matplotlib à sauvegarder
    :param artifact_name: le nom de l'artefact pour MLflow (inclure ".png")
    """
    # Utilisation d'un buffer en mémoire
    buffer = BytesIO()
    figure.savefig(buffer, format="png")
    buffer.seek(0)
    
    # Sauvegarder dans MLflow à partir du buffer
    with open(artifact_name, "wb") as f:
        f.write(buffer.getvalue())
    mlflow.log_artifact(artifact_name)

In [ ]:
 # Calcul et stockage des métriques dans un dictionnaire
from sklearn.metrics import precision_score, recall_score, f1_score, roc_curve, auc, log_loss
all_validation_metrics = []   
# Évaluation du modèle sur le set de validation en mini-batches
batch_size = 8  # Taille réduite pour l'évaluation
num_batches = len(val_input_ids) // batch_size
all_predictions = []
# Initialisation du dictionnaire des métriques
metrics_dict = {}
all_probs = []
# Initialisation d'une liste pour stocker les log loss par batch (optionnel, pour analyse batch-wise)
log_losses = []

for i in range(num_batches):
    # Obtenir un batch de validation
    batch_input_ids = val_input_ids[i * batch_size : (i + 1) * batch_size]
    batch_attention_masks = val_attention_masks[i * batch_size : (i + 1) * batch_size]
    batch_labels = val_labels[i * batch_size : (i + 1) * batch_size]  # Récupérer les labels correspondants


    # Calcul des logits pour le batch
    batch_logits = model(
        input_ids=batch_input_ids,
        attention_mask=batch_attention_masks,
        training=False
    ).logits

    # Stocker les prédictions
    batch_predictions = tf.argmax(batch_logits, axis=1)
    all_predictions.append(batch_predictions)
    
     # Convertir logits en probabilités
    batch_probabilities = tf.nn.softmax(batch_logits, axis=1)[:, 1]  # Probabilités pour la classe 1
    all_probs.append(batch_probabilities)
    
    # Calcul du log loss pour ce batch (en explicitant les classes [0, 1])
    batch_log_loss = log_loss(batch_labels, batch_probabilities, labels=[0, 1])
    log_losses.append(batch_log_loss)


# Concaténer toutes les prédictions
all_predictions = tf.concat(all_predictions, axis=0)

# Concaténer toutes les probabilités
all_probs = tf.concat(all_probs, axis=0).numpy()



# Calcul du log loss total sur l'ensemble des données
# total_log_loss = log_loss(val_labels[:len(all_probs)], all_probs, labels=[0, 1])
# validation_metrics["Validation Log Loss"] = round(total_log_loss, 3)
# print(f"Validation Log Loss: {total_log_loss:.4f}")

# # Tracer le log loss par batch
# plt.figure(figsize=(8, 6))
# plt.plot(range(len(log_losses)), log_losses, marker='o', label="Log Loss par batch")
# plt.axhline(total_log_loss, color='red', linestyle='--', label=f"Log Loss total ({total_log_loss:.4f})")
# plt.xlabel("Index des batches")
# plt.ylabel("Log Loss")
# plt.title("Logarithmic Loss par Batch")
# plt.legend()
# log_plot_to_mlflow(plt.gcf(), "log_loss_batches.png")
# plt.show()

# # Ajout du log loss dans MLflow
# mlflow.log_metric("val_log_loss", total_log_loss)



# Calcul de la courbe ROC
fpr, tpr, thresholds = roc_curve(val_labels[:len(all_probs)], all_probs)
roc_auc = auc(fpr, tpr)

# Tracer la courbe ROC
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f"ROC curve (AUC = {roc_auc:.2f})")
plt.plot([0, 1], [0, 1], 'k--', label="Random guess")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Bert ROC Curve - Validation Set")
plt.legend(loc="lower right")
log_plot_to_mlflow(plt.gcf(), "roc_curve_val.png")
plt.close()



# Calcul de l'accuracy
accuracy = tf.reduce_mean(tf.cast(all_predictions == val_labels[:len(all_predictions)], dtype=tf.float32))
print(f"Validation Accuracy: {accuracy.numpy():.4f}")

validation_metrics = {
    "Validation Accuracy": round(float(accuracy.numpy()), 3),
    "Validation ROC AUC": round(roc_auc, 3),
    "Validation Precision": round(precision_score(val_labels[:len(all_predictions)], all_predictions.numpy()), 3),
    "Validation Recall": round(recall_score(val_labels[:len(all_predictions)], all_predictions.numpy()), 3),
    "Validation F1": round(f1_score(val_labels[:len(all_predictions)], all_predictions.numpy()), 3)
}

metrics_dict.update(validation_metrics)
# Log de l'accuracy finale de validation et des autres métriques dans MLflow
mlflow.log_metrics(metrics_dict)

# Log de l'accuracy finale de validation dans MLflow
mlflow.log_metric("val_accuracy", accuracy.numpy())

# Enregistrement du modèle
mlflow.keras.log_model(model, "bert_model")

run_id = mlflow.active_run().info.run_id
result = mlflow.register_model(
    model_uri=f"runs:/{run_id}/model",
    name=f"bert-base-uncased"
)

# Construire le lien MLflow correspondant (en supposant que vous avez l'URL de votre serveur MLflow)
active_run = mlflow.active_run()
run_id = active_run.info.run_id
#run_name = active_run.data.tags.get("mlflow.runName")  # Obtenir le nom du run depuis les tags

# Récupérer l'ID de l'expérience active
experiment_id = active_run.info.experiment_id

# Définir l'URL de votre serveur MLflow
mlflow_server_url = "http://localhost:5000"  # Remplacez par l'URL réel de votre serveur MLflow

# Construire le lien complet vers le run dans l'interface MLflow
run_link = f"{mlflow_server_url}/#/experiments/{experiment_id}/runs/{run_id}"

# Ajouter les métriques de validation à la liste
all_validation_metrics.append({
    "run_name": f"bert-base-uncased-trained",
    **validation_metrics,
    "run_id": run_link
})
mlflow.log_table(pd.DataFrame(all_validation_metrics), "3-bert-base-uncased.json")
mlflow.end_run()

2024/11/23 09:50:07 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.


Validation Accuracy: 0.8125


/opt/venv/lib/python3.11/site-packages/transformers/generation/tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)
2024/11/23 09:50:13 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpjf0iw5im/model, flavor: keras). Fall back to return ['keras==3.6.0']. Set logging level to DEBUG to see the full traceback. 
2024/11/23 09:50:13 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'bert-base-uncased' already exists. Creating a new version of this model...
2024/11/23 09:50:13 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: bert-base-uncased, version 11
Cre

In [18]:
display(validation_metrics)

{'Validation Accuracy': 0.812,
 'Validation ROC AUC': 0.89,
 'Validation Precision': 0.787,
 'Validation Recall': 0.852,
 'Validation F1': 0.818}

In [19]:
display(pd.DataFrame(metrics_dict, index=[0]))

,Validation Accuracy,Validation ROC AUC,Validation Precision,Validation Recall,Validation F1
0,0.812,0.89,0.787,0.852,0.818


# Analyse Globale

## Points Forts
- Le modèle montre une **bonne capacité de généralisation** avec :
  - Une **Accuracy** respectable de 81.2%.
  - Un **AUC-ROC élevé** de 0.89, indiquant une bonne capacité à distinguer les classes.
- Le **Rappel élevé (85.2%)** montre que le modèle détecte efficacement les vrais positifs, 
  ce qui est souvent crucial dans des applications critiques.

## Points d'Amélioration
- La **Précision (78.7%)** est légèrement inférieure au Rappel, ce qui suggère la présence 
  de quelques **faux positifs**.

## Questions à Explorer
1. **Pourquoi un déséquilibre entre précision et rappel ?**
   - Le modèle semble favoriser le rappel (capturer les vrais positifs) au détriment de la précision (Tweet neutre?).
2. **Quel est l'impact métier des faux positifs et des faux négatifs ?**
   - Comprendre les coûts spécifiques de ces erreurs est crucial pour affiner le modèle.

---

# Recommandations

1. **Inspecter la matrice de confusion :**
   - Cela permettra de mieux comprendre les types d'erreurs commises 
     (faux positifs et faux négatifs).

2. **Ajuster le seuil de classification :**
   - Modifier le seuil peut équilibrer la Précision et le Rappel selon les priorités métier.

3. **Tester sur d'autres métriques métier :**
   - Par exemple, le coût pondéré des erreurs ou le taux de faux positifs 
     peut être plus pertinent pour des besoins spécifiques.

4. **Analyse approfondie des erreurs :**
   - Étudier les tweets mal classifiés pour identifier des tendances 
     ou des motifs que le modèle ne capte pas correctement.


# Conclusion

Le modèle **`bert-base-uncased`** a montré des performances prometteuses sur notre jeu de données, surpassant les résultats obtenus avec certains modèles spécialisés comme ceux adaptés à Twitter. Toutefois, la gestion des faux positifs et l'équilibrage entre Précision et Rappel nécessitent une attention particulière. Une étude plus poussée des erreurs et des ajustements de seuil pourraient encore améliorer ses performances et sa pertinence dans le contexte métier.